Université Paul Sabatier

EIMAB3H1 - Analyse et exploitation de données

Enseignant : **José G. Moreno**

## TP 2. Extraction d'information sur documents textuels (2/3)
III - L'usage de méthodes statistiques
Les règles peuvent nous rapporter des informations fausses ou ignorer certains usages (le dernier est le
plus fréquent). Les méthodes d'apprentissage statistiques sont couramment utilisés pour identifier
correctement les entités nommées. Pour le faire, ces méthodes utilisent plusieurs informations comme
le contexte ou l'information syntactique dans laquelle un mot apparaît. L'avantage de ces méthodes est
qu'ils n'utilisent pas des règles prédéfinies et ils s'adaptant plus facilement à des nouveaux documents.
Leur principal inconvénient est le coût de générer des annotations fiables pour l'étape d'apprentissage.

### I - Utilisation de méthodes statistiques
Nous allons utiliser la méthode de reconnaissances d’entités nommées implémenté sur nltk. Pour le
faire, nous allons utiliser des phrases dans lesquelles au moins un pays est mentionné. Voici le code
pour récupérer la liste de pays (comme dans le TP1) :


In [ ]:
!wget https://dumps.wikimedia.org/simplewiki/20200901/simplewiki-20200901-pages-articles.xml.bz2
!bzip2 -d simplewiki-20200901-pages-articles.xml.bz2
! echo "<mediawiki>" > simplewiki-20200901-pages-articles.xml.clean
! tail -n +2 simplewiki-20200901-pages-articles.xml >> simplewiki-20200901-pages-articles.xml.clean

--2020-11-06 08:44:13--  https://dumps.wikimedia.org/simplewiki/20200901/simplewiki-20200901-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179777498 (171M) [application/octet-stream]
Saving to: ‘simplewiki-20200901-pages-articles.xml.bz2’

simplewiki-20200901 100%[===================>] 171.45M  4.69MB/s    in 36s     

2020-11-06 08:44:49 (4.77 MB/s) - ‘simplewiki-20200901-pages-articles.xml.bz2’ saved [179777498/179777498]



In [ ]:
import pandas as pd
import xml.etree.ElementTree as et

def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    The first element of df_cols is supposed to be the identifier 
    variable, which is an attribute of each node element in the 
    XML data; other features will be parsed from the text content 
    of each sub-element. 
    """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
      if node.find(df_cols[0]) is not None:
        res = [node.find(df_cols[0]).text]
        for el in df_cols[1:]: 
            if node is not None and node.find('revision',).find(el,) is not None:
                res.append(node.find('revision',).find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

In [ ]:
df = parse_XML("simplewiki-20200901-pages-articles.xml.clean", ["title", "text"])
df

,title,text
0,April,{{monththisyear|4}}\n'''April''' is the fourth...
1,August,{{monththisyear|8}}\n'''August''' (Aug.) is th...
2,Art,[[Category:Art| ]]\n[[Category:Non-verbal comm...
3,A,{{about| the first [[letter]] in the [[alphabe...
4,Air,{{Simplify|date=February 2020}}\n[[Image:Kawas...
...,...,...
319768,Workers and Peasants Party,Bhagat sing was the grastest Second freedom fi...
319769,Vladislav Krapivin,[[File:Krapivin 2006.jpg|right|200px]]\n'''Vla...
319770,Tomomi Inada,{{Infobox officeholder\n|name = Tomo...
319771,Shigeru Ishiba,{{Infobox officeholder\n|name = Shig...


Nous allons accéder à la page  Wikipédia « List of countries » qui contient la liste des pays :

In [ ]:
print(df[df['title']=='List of countries']['text'].values)

['This is a list of [[sovereign state]]s.\n\n{| class="sortable wikitable" style="background:white; text-align:left;"\n|-\n! style="width:35%" |Common and formal names\n! style="width:12.5%;" |Membership within the [[United Nations System|UN System]]{{efn|This column indicates whether or not a state is a member of the [[United Nations]]. It also indicates which non-member states participate in the [[United Nations System]] through membership in the [[International Atomic Energy Agency]] or one of the [[List of specialized agencies of the United Nations|specialized agencies of the United Nations]]. All United Nations members belong to at least one specialized agency and are parties to the statute of the [[International Court of Justice]].}}\n! style="width:12.5%;" |Sovereignty dispute{{efn|This column indicates whether or not a state is the subject of a major sovereignty dispute. Only states whose entire sovereignty is disputed by another state are listed.}}\n! class="unsortable" |Furth

Voici la liste final dans une variable de type [set](https://docs.python.org/2/library/sets.html)

In [ ]:
import re

s = str(df[df['title']=='List of countries']['text'].values)
pattern = re.compile(r"\[\[([A-Za-z0-9_ ]+)\]\]")
my_countries = set([x for x in pattern.findall(s)])
my_countries

{'Abkhazia',
 'Abyei Area',
 'Aceh',
 'Adjara',
 'African Union',
 'Akrotiri and Dhekelia',
 'Ashmore and Cartier Islands',
 'Australian Antarctic Territory',
 'Autonomous Region in Muslim Mindanao',
 'Azores',
 'BBC News',
 'Bajo Nuevo Bank',
 'Baker Island',
 'Barbuda',
 'Belarus',
 'Bhutan',
 'Bonaire',
 'Bouvet Island',
 'China',
 'China and the United Nations',
 'Chinese Civil War',
 'Chinese Taipei',
 'Clipperton Island',
 'Commonwealth of Nations',
 'Commonwealth realm',
 'Communist Party of China',
 'Compact of Free Association',
 'Comprehensive Peace Agreement',
 'Constitution of Iceland',
 'Coral Sea Islands Territory',
 'Crown dependencies',
 'Cyprus dispute',
 'Dominion',
 'East Jerusalem',
 'Easter Island',
 'Economic Cooperation Organization',
 'Elizabeth II',
 'England',
 'European Union',
 'Federal government of the United States',
 'Federated state',
 'Foreign relations of China',
 'Foreign relations of Cyprus',
 'Foreign relations of North Korea',
 'Foreign relations 

Une fois la liste est produite, nous pouvons rechercher des phrases avec des mentions de la liste. Voici
le code pour rechercher des phrases qui contient des mentions des pays :

In [ ]:
def findPhrases(text,l):
 parts = str(text).split("\n")
 sentences = []
 for part in parts:
   for val in re.finditer('[A-Z](\w| |\,)+\.', part):
     if len(val.group(0).split(" "))<10:
       continue
     for country in l:
       if country in val.group(0):
         sentences.append(val.group(0))
         break
 return sentences

sentences = []
for page in list(df['text'])[:1000]:
  sentences.extend(findPhrases(page,my_countries))

sentences[:15]

['Elizabeth II greets NASA GSFC employees, May 8, 2007 edit.',
 'Italy, and the Romans later modified the Etruscan alphabet for their own language.',
 'In 2013, almost 60 years later, Turing received a posthumous Royal Pardon from Queen Elizabeth II.',
 'They collected hundreds of plants to take back to England.',
 'England needed to find a new and less populated place.',
 'By the 1780s the gaols of England were so full that convicts were often chained up in rotting old ships.',
 'The government decided to make a settlement in New South Wales and send some of the convicts there.',
 'Spanish Merino sheep had been brought to Sydney, and by 1820, farmers were raising fat lambs for meat and also sending fine wool back to the factories of England.',
 'While the settlement was growing in New South Wales, it was also growing in Tasmania.',
 'The climate in Tasmania was more like that in England, and farmers found it easy to grow crops there.',
 'The gold rushes of New South Wales and Victoria

#### NLTK
NLKT est une librarie pour la reconnaisance d'entités nommées. Pour utiliser une des méthodes implémentées sur nltk, il suffit d’utiliser le code ci-dessous

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
print(nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize("May has also dealt with the war in Iraq and Syria."))))

(S
  May/NNP
  has/VBZ
  also/RB
  dealt/VBN
  with/IN
  the/DT
  war/NN
  in/IN
  (GPE Iraq/NNP)
  and/CC
  (GSP Syria/NNP)
  ./.)


Notez que d’abord il faut trouver les « tokens », puis faire l’étiquetage morphe-syntaxique (POS
tagging) et finalement la reconnaissances d’entités nommées. Regardez le support du cours pour des
explications. 

1) Expliquez la fonctionalité et l'interêt de faire des lignes de code ci-dessous 

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

on telecharge les packages 

'Punkt'  permet le decoupage de phrase comme un split mais plus perfomant[ il par exemple sait que les points de M. Smith et Johann S. Bach 
    ... ne marquent pas les limites des phrases. Et parfois, les phrases 
    ... peuvent commencer par des mots non capitalisés. i est un bon 
    nom de 
variable . ]

'averaged_perceptron_tagger' est le processus qui consiste à associer aux mots d'un texte les informations grammaticales correspondantes (Étiquetage)


'maxent_ne_chunker' contient deux blocs d'entités nommées anglais pré-entraînés formés sur un corpus ACE. Il chargera un objet nltk.chunk.named_entity.NEChunkParser et il sera utilisé par la fonction nltk.ne_chunk ().

'word' il decoupe les phrases en plusieurs mots 

In [ ]:
print(nltk.ne_chunk(nltk.pos_tag("May has also dealt with the war in Iraq and Syria.".split())))

(S
  May/NNP
  has/VBZ
  also/RB
  dealt/VBN
  with/IN
  the/DT
  war/NN
  in/IN
  (GPE Iraq/NNP)
  and/CC
  Syria./NNP)


2) Pourquoi le code marche sachant que la fonctionne ``` nltk.word_tokenize ``` n'est pas appelée ? Expliquez les différences entre les résultats obtenues par ``` nltk.word_tokenize ``` et ``` split() ```.

In [ ]:
print(nltk.word_tokenize("May has also dealt with the war in Iraq and Syria."))
print(("May has also dealt with the war in Iraq and Syria.").split())

['May', 'has', 'also', 'dealt', 'with', 'the', 'war', 'in', 'Iraq', 'and', 'Syria', '.']
['May', 'has', 'also', 'dealt', 'with', 'the', 'war', 'in', 'Iraq', 'and', 'Syria.']


ce code marche car tokenize et split on la meme fonctionnaalite qui est le decoupage d'une phrase en plusieurs mots . 
mais tokenize est plus perfomant car il est entrainé à mieux reconnaitre la limite d'une entité.


3) Pourquoi le code


In [ ]:
print(nltk.ne_chunk(nltk.word_tokenize("May has also dealt with the war in Iraq and Syria.")))

IndexError: ignored

ne marche pas (un erreur est généré) ? Pourquoi faut-il utiliser ``` 

---

nltk.pos_tag ``` ?

ltk.ne_chunk est tjr associé à la fonction nltk.pos_tag car  il prend en parametre  tagged_tokens qui est un tuple de l'entite et son ettiquette (tag).


**fonction ne_chunk**


ne_chunk ( tagged_tokens ,  binaire = False ):

  si  binaire : 
        chunker_pickle  =  _BINARY_NE_CHUNKER 
    autre : 
        chunker_pickle  =  _MULTICLASS_NE_CHUNKER 
    chunker  =  charge ( chunker_pickle ) 
    retour  chunker . analyser ( tagged_tokens )

4) Quel type d'algorithme de reconnaissances d’entités nommées 

---

(classification, séquences, etc.) qui est implémenté par la méthode ne_chunk ?
Justifiez la réponse.

cest un programme classifeur , car il  fragmente la liste donnée de jetons étiquetés.



5) Modifiez le code pour pouvoir appliquer la reconnaissance d’entités à toutes les phrases dans la variable ``` sentences ```. Dans quels cas ne_chunk ne marche pas ?
```
print(nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize("May has also dealt with the war in Iraq and Syria."))))

```
 

In [ ]:
for sentence in sentences :
  print(nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sentence))))

ne_chunk ne marche pas qd 

6) Utilisez ``` ne_chuck ``` pour faire la reconnaissance d’entités nommées sur des pages complètes.
Comparez les résultats entre pages de nature différent (Pays vs Personne, par exemple France vs Barack Obama). Trouver vous des différences
significatives entre les frequences d'entités par type ?

In [ ]:
Obama=df[df['title']=='Barack Obama']['text'].values
France=df[df['title']=='France']['text'].values

In [ ]:
def findPhrases2(text):
 parts = str(text).split("\n")
 sentences = []
 for part in parts:
   for val in re.finditer('[A-Z](\w| |\,)+\.', part):
     if len(val.group(0).split(" "))<10: 
       sentences.append(val.group(0))

 return sentences


In [ ]:
ObamaSentences=findPhrases2(str(Obama))
ObamaSentences
compteurO=0
DictObama={}
for sentence in ObamaSentences:
  val=nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sentence))).pos()
  for valeur in val :
    compteurO+=1
    if (valeur[1] not in DictObama):
      DictObama[valeur[1]]=1
    else:
      DictObama[valeur[1]]+=1

print(DictObama)

for i in DictObama.keys():
  DictObama[i]=DictObama[i]*100/compteurO
print(DictObama)


In [ ]:
DictFrance={}
FranceSentences=findPhrases2(str(France))
compteurF=0
for sentence in FranceSentences:
  val=nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sentence))).pos()
  for valeur in val :
    compteurF+=1
    if (valeur[1] not in DictFrance):
      DictFrance[valeur[1]]=1
    else:
      DictFrance[valeur[1]]+=1

print(DictFrance)
for i in DictFrance.keys():
  DictFrance[i]=DictFrance[i]*100/compteurF
print(DictFrance)


on remarque que la frequence de 

7) Utilisez les instructions sur https://stanfordnlp.github.io/stanza/ pour
installer/utiliser le système de reconnaissances d’entités nommées de stanford. Appliquez le modèle sur les phrases que vous avez
 dans la variable ``` sentences ``` et comparez les résultats obtenus avec ``` ne_chunck ```  de NLTK. Trouvez-vous des
différences ? Quel facteur a peut les générer ?

In [ ]:
!pip install stanza

In [ ]:
import stanza
stanza.download('en') # download English model


In [ ]:
nlp = stanza.Pipeline('en') # initialize English neural pipeline


In [ ]:
DictObama2={}
compteur=0
for sentence in ObamaSentences:
  doc=nlp(sentence)
  for val in doc.entities:
    compteur+=1
    if val.type not in DictObama2:
        DictObama2[val.type]=1
    else:
        DictObama2[val.type]+=1

for i in DictObama2.keys():
  DictObama2[i]=DictObama2[i]*100/compteur


print("methode chunk",DictObama)
print("methode STANZA",DictObama2)



8) Après vos expériences avec ``` ne_chunk ``` et ``` Stanza ```. Listez les gros lignes pour
implémenter un système statistique pour le NER.